In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    f1_score, 
    precision_score, 
    recall_score, 
    accuracy_score, 
    classification_report
)
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle

In [2]:
data = pd.read_pickle("/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/cubert_embeddings/processed_embeddings/cubert_embed_line_by_line_avg.pkl")
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,0.531472,-0.101989,-0.314334,0.647844,-0.035478,-0.714266,0.872624,-0.409889,0.890056,0.453202,...,0.209240,-0.288426,-0.539697,0.276517,0.440467,1.152947,-0.153769,1,8077,critical
1,0.418732,-0.778774,-0.598116,0.924988,-0.092339,-0.632556,0.891250,-0.204653,1.435295,0.099128,...,-0.480547,-0.364436,-1.308979,1.307274,-0.088777,1.309955,-0.166052,1,5553,critical
2,0.006259,-0.255758,-0.488272,0.934097,-0.192474,-0.773479,0.521129,-0.148380,0.954459,0.330002,...,0.690031,-0.331558,-0.138192,-0.551222,0.302782,-0.032620,0.666646,1,9341,critical
3,0.546839,-0.356427,-0.596382,0.627335,-0.049085,-0.261811,0.887523,-0.361324,0.746271,0.477907,...,0.093244,0.153900,-0.112701,0.416357,0.170588,0.590961,-0.729293,1,10419,critical
4,0.531472,-0.101989,-0.314334,0.647844,-0.035478,-0.714266,0.872624,-0.409889,0.890056,0.453202,...,0.209240,-0.288426,-0.539697,0.276517,0.440467,1.152947,-0.153769,1,12232,critical


In [3]:
# Separate features and labels
X = data[[col for col in data.columns if col.startswith("emb_")]].values  # Select embedding columns
y = data['label'].values 

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define classifiers
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
bagging = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=100, random_state=42)  
xgboost = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Add the classifiers to a dictionary
classifiers = {
    "Random Forest": random_forest,
    "Bagging": bagging,
    "XGBoost": xgboost
}

In [4]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
results = {}

# Perform cross-validation for each classifier
for clf_name, clf in classifiers.items():
    metrics_summary = {
        "f1_micro": [],
        "f1_macro": [],
        "accuracy": [],
        "precision_macro": [],
        "recall_macro": []
    }
    
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Split the training data into training and validation folds
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        # Train the classifier
        clf.fit(X_train_fold, y_train_fold)
        
        # Make predictions on the validation set
        y_pred = clf.predict(X_val_fold)
        
        # Calculate metrics
        metrics_summary["f1_micro"].append(f1_score(y_val_fold, y_pred, average='micro'))
        metrics_summary["f1_macro"].append(f1_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["accuracy"].append(accuracy_score(y_val_fold, y_pred))
        metrics_summary["precision_macro"].append(precision_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["recall_macro"].append(recall_score(y_val_fold, y_pred, average='macro'))
    
    # Store mean metrics for the classifier
    results[clf_name] = {metric: np.mean(scores) for metric, scores in metrics_summary.items()}

# Print cross-validation results
print("\nCross-Validation Results:")
for clf_name, metrics in results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")


Cross-Validation Results:

Random Forest Metrics:
F1_micro: 0.8228
F1_macro: 0.7570
Accuracy: 0.8228
Precision_macro: 0.7808
Recall_macro: 0.7421

Bagging Metrics:
F1_micro: 0.8228
F1_macro: 0.7490
Accuracy: 0.8228
Precision_macro: 0.7877
Recall_macro: 0.7290

XGBoost Metrics:
F1_micro: 0.8263
F1_macro: 0.7659
Accuracy: 0.8263
Precision_macro: 0.7825
Recall_macro: 0.7542


In [5]:
# Final evaluation on the test set
final_results = {}

for clf_name, clf in classifiers.items():
    # Train the classifier on the entire training set
    clf.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_test = clf.predict(X_test)
    
    # Calculate test metrics
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    accuracy_test = accuracy_score(y_test, y_pred_test)
    report_test = classification_report(y_test, y_pred_test)
    
    # Save final metrics for the classifier
    final_results[clf_name] = {
        "f1_micro": f1_micro_test,
        "f1_macro": f1_macro_test,
        "precision_macro": precision_test,
        "recall_macro": recall_test,
        "accuracy": accuracy_test,
        "classification_report": report_test
    }

# Print final test results
print("\nFinal Test Results:")
for clf_name, metrics in final_results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        if metric != "classification_report":
            print(f"{metric.capitalize()}: {value:.4f}")
    print(f"Classification Report:\n{metrics['classification_report']}")



Final Test Results:

Random Forest Metrics:
F1_micro: 0.8179
F1_macro: 0.7576
Precision_macro: 0.7695
Recall_macro: 0.7484
Accuracy: 0.8179
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       814
           1       0.68      0.60      0.64       295

    accuracy                           0.82      1109
   macro avg       0.77      0.75      0.76      1109
weighted avg       0.81      0.82      0.81      1109


Bagging Metrics:
F1_micro: 0.8133
F1_macro: 0.7463
Precision_macro: 0.7656
Recall_macro: 0.7334
Accuracy: 0.8133
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.90      0.88       814
           1       0.68      0.56      0.62       295

    accuracy                           0.81      1109
   macro avg       0.77      0.73      0.75      1109
weighted avg       0.81      0.81      0.81      1109


XGBoost Metrics:
F1_micro: 0.8079
F1_macro: 

In [7]:
# Save results to a pickle file
output_path = "/Users/mac/Desktop/Code_Smell_Detection/data_class/results/cubert/result_cubert_avg.pkl"
with open(output_path, "wb") as f:
    pickle.dump({"cross_val_results": results, "final_results": final_results}, f)
print(f"\nResults saved to {output_path}")



Results saved to /Users/mac/Desktop/Code_Smell_Detection/data_class/results/cubert/result_cubert_avg.pkl
